# MIGHTEE-HI Data: Velocity Function
***
By Wanga Mulaudzi 

<b>Date:</b> 1 August 2021
<br>
<b>Affiliation:</b> Department of Astronomy, University of Cape Town, Private Bag X3, Rondebosch 7701, South Africa
<br>
<b>Contact:</b> MLDWAN001@myuct.ac.za
<br>
<b>Ilifu Jupyter Kernel:</b> Python 3

This notebook will construct a velocity function from MIGHTEE-HI data and make comparisons with Zwaan 2010 and Papastergis 2011.

In [33]:
# Import statements
from astropy.io import ascii
from astropy import units as u
from astropy import constants as const
from astropy.cosmology import LambdaCDM
import collections
import json
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoLocator, AutoMinorLocator, MultipleLocator)
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as mtick
import numpy as np
from numpy import log, exp, pi
import os
import pylab as pl
import scipy as sp
#import scipy.stats
import shutil
import datetime
import matplotlib.dates as mdates
from matplotlib.transforms import Transform

In [34]:
# # Initiate parameters for plotting
# pl.rc('axes',titlesize='large')
# pl.rc('text', usetex=True)
# pl.rc('font', **{'family':'serif','size':20})
# pl.rc('axes', labelsize=16)
# pl.rc('xtick',labelsize=16)
# pl.rc('ytick',labelsize=16)

# plt.rcParams['mathtext.fontset'] = 'custom'
# plt.rcParams['mathtext.rm'] = 'serif'
# plt.rcParams['font.family'] = 'serif'

## Paths to text files and detection lists

The paths below point to where the outputs of step 1, 2 and 3 are.

In [35]:
'''
COSMOS123
'''
# COSMOS123 1310
cosmos123_1310_path = '/users/wanga/mightee/analysis/COSMOS_1310/'
cosmos123_1310_dets = ascii.read('/users/wanga/mightee/source_finding/COSMOS/COSMOS123_1310_catalogue_notes.txt')

# COSMOS123 1330
cosmos123_1330_path = '/users/wanga/mightee/analysis/COSMOS_1330/'
cosmos123_1330_dets = ascii.read('/users/wanga/mightee/source_finding/COSMOS/COSMOS123_1330_catalogue_notes.txt')

# COSMOS123 1380
cosmos123_1380_path = '/users/wanga/mightee/analysis/COSMOS_1380/'
cosmos123_1380_dets = ascii.read('/users/wanga/mightee/source_finding/COSMOS/COSMOS123_1380_catalogue_notes.txt')

'''
XMMLSS12
'''
# XMMLSS12 1310
xmmlss12_1310_path = '/users/wanga/mightee/analysis/XMMLSS12_1310/'
xmmlss12_1310_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS12/XMMLSS12_1310_catalogue_notes.txt')

# XMMLSS12 1330
xmmlss12_1330_path = '/users/wanga/mightee/analysis/XMMLSS12_1330/'
xmmlss12_1330_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS12/XMMLSS12_1330_catalogue_notes.txt')

# XMMLSS12 1380
xmmlss12_1380_path = '/users/wanga/mightee/analysis/XMMLSS12_1380/'
xmmlss12_1380_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS12/XMMLSS12_1380_catalogue_notes.txt')

'''
XMMLSS13
'''
# XMMLSS13 1310
xmmlss13_1310_path = '/users/wanga/mightee/analysis/XMMLSS13_1310/'
xmmlss13_1310_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS13/XMMLSS13_1310_catalogue_notes.txt')

# XMMLSS13 1330
xmmlss13_1330_path = '/users/wanga/mightee/analysis/XMMLSS13_1330/'
xmmlss13_1330_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS13/XMMLSS13_1330_catalogue_notes.txt')

# XMMLSS13 1380
xmmlss13_1380_path = '/users/wanga/mightee/analysis/XMMLSS13_1380/'
xmmlss13_1380_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS13/XMMLSS13_1380_catalogue_notes.txt')

'''
XMMLSS14
'''
# XMMLSS14 1310
xmmlss14_1310_path = '/users/wanga/mightee/analysis/XMMLSS14_1310/'
xmmlss14_1310_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS14/XMMLSS14_1310_catalogue_notes.txt')

# XMMLSS14 1330
xmmlss14_1330_path = '/users/wanga/mightee/analysis/XMMLSS14_1330/'
xmmlss14_1330_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS14/XMMLSS14_1330_catalogue_notes.txt')

# XMMLSS14 1380
xmmlss14_1380_path = '/users/wanga/mightee/analysis/XMMLSS14_1380/'
xmmlss14_1380_dets = ascii.read('/users/wanga/mightee/source_finding/XMMLSS14/XMMLSS14_1380_catalogue_notes.txt')

## Reading in the $W_{50}$ values, peak flux, redshift and distance

In [204]:
# Type of w50s to read in
w50_type = 'sdv'

mightee_w50_flux_z_D_himass_vsys = []
cosm_w50_flux_z_D_himass_vsys = []
xmmlss_w50_flux_z_D_himass_vsys = []
xmmlss12_w50_flux_z_D_himass_vsys = []
xmmlss13_w50_flux_z_D_himass_vsys = []
xmmlss14_w50_flux_z_D_himass_vsys = []

# Detections outside effective area
mightee_w50_flux_z_D_himass_vsys_excl = []
mightee_mstellar_excl = []
cosm_w50_flux_z_D_himass_vsys_excl = []
xmmlss_w50_flux_z_D_himass_vsys_excl = []
xmmlss12_w50_flux_z_D_himass_vsys_excl = []
xmmlss13_w50_flux_z_D_himass_vsys_excl = []
xmmlss14_w50_flux_z_D_himass_vsys_excl = []

In [205]:
# Detections outside the effective area
x12_eff = ['MGTH_J021831.4-054452', 'MGTH_J021529.1-051859']
x13_eff = ['MGTH_J022230.9-040742', 'MGTH_J022123.0-040259', 'MGTH_J022028.5-053705', 'MGTH_J022111.7-040445']
x14_eff = ['MGTH_J022454.9-052548', 'MGTH_J022233.1-054249', 'MGTH_J022512.4-040915', 'MGTH_J022541.9-051835', 'MGTH_J022537.3-052542', 
           'MGTH_J022628.5-050905', 'MGTH_J022611.0-044104', 'MGTH_J022702.5-043531', 'MGTH_J022244.3-035807', 'MGTH_J022556.7-041857', 'MGTH_J022325.3-053839']
cosm_eff = ['MGTH_J100222.8+024519', 'MGTH_J100103.7+013055', 'MGTH_J100313.9+022031', 'MGTH_J095720.6+015507', 'MGTH_J100313.5+015343', 'MGTH_J100212.1+030312', 
            'MGTH_J100035.1+011433', 'MGTH_J100048.1+011414', 'MGTH_J100357.1+022505', 'MGTH_J100352.4+022534', 'MGTH_J095822.5+013505', 'MGTH_J095728.7+030509', 
            'MGTH_J095727.3+015221', 'MGTH_J100031.6+031219', 'MGTH_J100315.4+020543', 'MGTH_J095829.1+014139', 'MGTH_J100050.5+012008', 'MGTH_J100306.5+015019']
mightee_eff = []

In [206]:
# Detections that need to be discarded
cosmos123_1310_discard = [34]
cosmos123_1330_discard = []
cosmos123_1380_discard = []

xmmlss12_1310_discard = []
xmmlss12_1330_discard = [27]
xmmlss12_1380_discard = [2]

xmmlss13_1310_discard = []
xmmlss13_1330_discard = [13, 34]
xmmlss13_1380_discard = []

xmmlss14_1310_discard = [16]
xmmlss14_1330_discard = [4]
xmmlss14_1380_discard = []

#### COSMOS123

In [207]:
# COSMOS123 1310
# Redshift text file
z_txt_file = cosmos123_1310_path+'masked_profiles_COSMOS_1310_output/COSMOS_1310_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = cosmos123_1310_path+'masked_profiles_COSMOS_1310_output/COSMOS_1310_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(cosmos123_1310_dets)):
    # Flux vs velocity text files
    prof_txt_file = cosmos123_1310_path+'masked_profiles_COSMOS_1310_output/COSMOS_1310_masked_detection'+str(cosmos123_1310_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = cosmos123_1310_path+'busy_function_COSMOS_1310_analysis_output/detection_'+str(cosmos123_1310_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(cosmos123_1310_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(cosmos123_1310_dets[i]['Detection'], 2*w50_data[1][3], np.log10(cosmos123_1310_dets[i]['MHI(1e9)']*1e9)))
    
    if (cosmos123_1310_dets[i]['Detection'] in cosmos123_1310_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (cosmos123_1310_dets[i]['Name'] in cosm_eff):
        if (2*w50_data[1][3] < 44):
            cosm_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(cosmos123_1310_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            cosm_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(cosmos123_1310_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        cosm_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        cosm_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

Detection 34: 438.77 km/s, HI mass = 9.19


In [208]:
# COSMOS123 1330
# Redshift text file
z_txt_file = cosmos123_1330_path+'masked_profiles_COSMOS_1330_output/COSMOS_1330_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = cosmos123_1330_path+'masked_profiles_COSMOS_1330_output/COSMOS_1330_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(cosmos123_1330_dets)):
    # Flux vs velocity text files
    prof_txt_file = cosmos123_1330_path+'masked_profiles_COSMOS_1330_output/COSMOS_1330_masked_detection'+str(cosmos123_1330_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = cosmos123_1330_path+'busy_function_COSMOS_1330_analysis_output/detection_'+str(cosmos123_1330_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(cosmos123_1330_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(cosmos123_1330_dets[i]['Detection'], 2*w50_data[1][3], np.log10(cosmos123_1330_dets[i]['MHI(1e9)']*1e9)))
    
    if (cosmos123_1330_dets[i]['Detection'] in cosmos123_1330_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (cosmos123_1330_dets[i]['Name'] in cosm_eff):
        if (2*w50_data[1][3] < 44):
            cosm_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(cosmos123_1330_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            cosm_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(cosmos123_1330_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        cosm_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        cosm_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

In [209]:
# COSMOS123 1380
# Redshift text file
z_txt_file = cosmos123_1380_path+'masked_profiles_COSMOS_1380_output/COSMOS_1380_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = cosmos123_1380_path+'masked_profiles_COSMOS_1380_output/COSMOS_1380_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(cosmos123_1380_dets)):
    # Flux vs velocity text files
    prof_txt_file = cosmos123_1380_path+'masked_profiles_COSMOS_1380_output/COSMOS_1380_masked_detection'+str(cosmos123_1380_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(cosmos123_1380_dets[i]['Detection'], 2*w50_data[1][3], np.log10(cosmos123_1380_dets[i]['MHI(1e9)']*1e9)))
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = cosmos123_1380_path+'busy_function_COSMOS_1380_analysis_output/detection_'+str(cosmos123_1380_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(cosmos123_1380_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    if (cosmos123_1380_dets[i]['Detection'] in cosmos123_1380_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (cosmos123_1380_dets[i]['Name'] in cosm_eff):
        if (2*w50_data[1][3] < 44):
            cosm_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(cosmos123_1380_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            cosm_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(cosmos123_1380_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        cosm_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        cosm_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], cosmos123_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

In [210]:
len(mightee_eff), len(mightee_w50_flux_z_D_himass_vsys)+len(mightee_w50_flux_z_D_himass_vsys_excl), len(cosm_w50_flux_z_D_himass_vsys)+len(cosm_w50_flux_z_D_himass_vsys_excl)

(78, 78, 78)

#### XMMLSS12

In [211]:
# XMMLSS12 1310
# Redshift text file
z_txt_file = xmmlss12_1310_path+'masked_profiles_XMMLSS12_1310_output/XMMLSS12_1310_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss12_1310_path+'masked_profiles_XMMLSS12_1310_output/XMMLSS12_1310_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss12_1310_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss12_1310_path+'masked_profiles_XMMLSS12_1310_output/XMMLSS12_1310_masked_detection'+str(xmmlss12_1310_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss12_1310_path+'busy_function_XMMLSS12_1310_analysis_output/detection_'+str(xmmlss12_1310_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss12_1310_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss12_1310_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss12_1310_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss12_1310_dets[i]['Detection'] in xmmlss12_1310_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss12_1310_dets[i]['Name'] in x12_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss12_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss12_1310_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss12_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss12_1310_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss12_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss12_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

In [212]:
# XMMLSS12 1330
# Redshift text file
z_txt_file = xmmlss12_1330_path+'masked_profiles_XMMLSS12_1330_output/XMMLSS12_1330_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss12_1330_path+'masked_profiles_XMMLSS12_1330_output/XMMLSS12_1330_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss12_1330_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss12_1330_path+'masked_profiles_XMMLSS12_1330_output/XMMLSS12_1330_masked_detection'+str(xmmlss12_1330_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss12_1330_path+'busy_function_XMMLSS12_1330_analysis_output/detection_'+str(xmmlss12_1330_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss12_1330_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss12_1330_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss12_1330_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss12_1330_dets[i]['Detection'] in xmmlss12_1330_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss12_1330_dets[i]['Name'] in x12_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss12_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss12_1330_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss12_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss12_1330_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss12_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss12_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

Detection 27: 456.49 km/s, HI mass = 8.74


In [213]:
# XMMLSS12 1380
# Redshift text file
z_txt_file = xmmlss12_1380_path+'masked_profiles_XMMLSS12_1380_output/XMMLSS12_1380_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss12_1380_path+'masked_profiles_XMMLSS12_1380_output/XMMLSS12_1380_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss12_1380_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss12_1380_path+'masked_profiles_XMMLSS12_1380_output/XMMLSS12_1380_masked_detection'+str(xmmlss12_1380_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss12_1380_path+'busy_function_XMMLSS12_1380_analysis_output/detection_'+str(xmmlss12_1380_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss12_1380_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss12_1380_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss12_1380_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss12_1380_dets[i]['Detection'] in xmmlss12_1380_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss12_1380_dets[i]['Name'] in x12_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss12_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss12_1380_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss12_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss12_1380_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss12_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss12_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss12_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

In [214]:
print(len(mightee_eff), len(mightee_w50_flux_z_D_himass_vsys)+len(mightee_w50_flux_z_D_himass_vsys_excl))
print(len(cosm_w50_flux_z_D_himass_vsys)+len(cosm_w50_flux_z_D_himass_vsys_excl)+len(xmmlss_w50_flux_z_D_himass_vsys)+len(xmmlss_w50_flux_z_D_himass_vsys_excl))

133 133
133


#### XMMLSS13

In [215]:
# XMMLSS13 1310
# Redshift text file
z_txt_file = xmmlss13_1310_path+'masked_profiles_XMMLSS13_1310_output/XMMLSS13_1310_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss13_1310_path+'masked_profiles_XMMLSS13_1310_output/XMMLSS13_1310_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss13_1310_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss13_1310_path+'masked_profiles_XMMLSS13_1310_output/XMMLSS13_1310_masked_detection'+str(xmmlss13_1310_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss13_1310_path+'busy_function_XMMLSS13_1310_analysis_output/detection_'+str(xmmlss13_1310_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss13_1310_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss13_1310_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss13_1310_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss13_1310_dets[i]['Detection'] in xmmlss13_1310_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss13_1310_dets[i]['Name'] in x13_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss13_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss13_1310_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss13_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss13_1310_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss13_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss13_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

Detection 21: 505.98 km/s, HI mass = 10.01


In [216]:
# XMMLSS13 1330
# Redshift text file
z_txt_file = xmmlss13_1330_path+'masked_profiles_XMMLSS13_1330_output/XMMLSS13_1330_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss13_1330_path+'masked_profiles_XMMLSS13_1330_output/XMMLSS13_1330_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss13_1330_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss13_1330_path+'masked_profiles_XMMLSS13_1330_output/XMMLSS13_1330_masked_detection'+str(xmmlss13_1330_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss13_1330_path+'busy_function_XMMLSS13_1330_analysis_output/detection_'+str(xmmlss13_1330_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss13_1330_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss13_1330_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss13_1330_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss13_1330_dets[i]['Detection'] in xmmlss13_1330_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss13_1330_dets[i]['Name'] in x13_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss13_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss13_1330_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss13_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss13_1330_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss13_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss13_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

Detection 34: 476.73 km/s, HI mass = 8.34


In [217]:
# XMMLSS13 1380
# Redshift text file
z_txt_file = xmmlss13_1380_path+'masked_profiles_XMMLSS13_1380_output/XMMLSS13_1380_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss13_1380_path+'masked_profiles_XMMLSS13_1380_output/XMMLSS13_1380_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss13_1380_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss13_1380_path+'masked_profiles_XMMLSS13_1380_output/XMMLSS13_1380_masked_detection'+str(xmmlss13_1380_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss13_1380_path+'busy_function_XMMLSS13_1380_analysis_output/detection_'+str(xmmlss13_1380_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss13_1380_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss13_1380_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss13_1380_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss13_1380_dets[i]['Detection'] in xmmlss13_1380_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss13_1380_dets[i]['Name'] in x13_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss13_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss13_1380_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss13_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss13_1380_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss13_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss13_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss13_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

In [218]:
print(len(mightee_eff), len(mightee_w50_flux_z_D_himass_vsys)+len(mightee_w50_flux_z_D_himass_vsys_excl))
print(len(cosm_w50_flux_z_D_himass_vsys)+len(cosm_w50_flux_z_D_himass_vsys_excl)+len(xmmlss_w50_flux_z_D_himass_vsys)+len(xmmlss_w50_flux_z_D_himass_vsys_excl))

193 193
193


#### XMMLSS14

In [219]:
# XMMLSS14 1310
# Redshift text file
z_txt_file = xmmlss14_1310_path+'masked_profiles_XMMLSS14_1310_output/XMMLSS14_1310_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss14_1310_path+'masked_profiles_XMMLSS14_1310_output/XMMLSS14_1310_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss14_1310_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss14_1310_path+'masked_profiles_XMMLSS14_1310_output/XMMLSS14_1310_masked_detection'+str(xmmlss14_1310_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss14_1310_path+'busy_function_XMMLSS14_1310_analysis_output/detection_'+str(xmmlss14_1310_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss14_1310_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss14_1310_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss14_1310_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss14_1310_dets[i]['Detection'] in xmmlss14_1310_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss14_1310_dets[i]['Name'] in x14_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss14_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss14_1310_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss14_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss14_1310_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss14_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss14_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1310_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

Detection 16: 469.42 km/s, HI mass = 8.62


In [220]:
# XMMLSS14 1330
# Redshift text file
z_txt_file = xmmlss14_1330_path+'masked_profiles_XMMLSS14_1330_output/XMMLSS14_1330_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss14_1330_path+'masked_profiles_XMMLSS14_1330_output/XMMLSS14_1330_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss14_1330_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss14_1330_path+'masked_profiles_XMMLSS14_1330_output/XMMLSS14_1330_masked_detection'+str(xmmlss14_1330_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss14_1330_path+'busy_function_XMMLSS14_1330_analysis_output/detection_'+str(xmmlss14_1330_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss14_1330_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss14_1330_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss14_1330_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss14_1330_dets[i]['Detection'] in xmmlss14_1330_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss14_1330_dets[i]['Name'] in x14_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss14_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss14_1330_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss14_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss14_1330_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss14_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss14_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1330_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

Detection 41: 465.71 km/s, HI mass = 9.11


In [221]:
# XMMLSS14 1380
# Redshift text file
z_txt_file = xmmlss14_1380_path+'masked_profiles_XMMLSS14_1380_output/XMMLSS14_1380_redshifts.txt'
z_data = np.genfromtxt(z_txt_file, delimiter=' ').T

# HI mass text file
himass_txt_file = xmmlss14_1380_path+'masked_profiles_XMMLSS14_1380_output/XMMLSS14_1380_masked_profile_HImasses.txt'
himass_data = np.genfromtxt(himass_txt_file, delimiter=' ').T
    
for i in range(len(xmmlss14_1380_dets)):
    # Flux vs velocity text files
    prof_txt_file = xmmlss14_1380_path+'masked_profiles_XMMLSS14_1380_output/XMMLSS14_1380_masked_detection'+str(xmmlss14_1380_dets[i]['Detection'])+'_profile_vel.txt'
    prof_data = np.genfromtxt(prof_txt_file, delimiter=' ').T
    
    # Busy function output
    # Read in the w50 text file for detection i
    w50_txt_file = xmmlss14_1380_path+'busy_function_XMMLSS14_1380_analysis_output/detection_'+str(xmmlss14_1380_dets[i]['Detection'])+'/med_b1b2xexpn_'+w50_type+'_results_detection_'+str(xmmlss14_1380_dets[i]['Detection'])+'.txt'
    w50_data = np.genfromtxt(w50_txt_file, delimiter=' ').T
    
    # Highest mass galaxies
    if 2*w50_data[1][3] > 422:
        print('Detection %i: %.2f km/s, HI mass = %.2f'%(xmmlss14_1380_dets[i]['Detection'], 2*w50_data[1][3], np.log10(xmmlss14_1380_dets[i]['MHI(1e9)']*1e9)))
    
    if (xmmlss14_1380_dets[i]['Detection'] in xmmlss14_1380_discard):
        # Discard detections with bad Busy Function fits
        continue
    if (xmmlss14_1380_dets[i]['Name'] in x14_eff):
        if (2*w50_data[1][3] < 44):
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss14_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss14_1380_dets[i]['Name'])
        else:
            # Append the w50, its error and the peak flux to the main list
            xmmlss_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            xmmlss14_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_w50_flux_z_D_himass_vsys_excl.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
            mightee_eff.append(xmmlss14_1380_dets[i]['Name'])
        continue
    elif (2*w50_data[1][3] < 44):
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss14_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([44.1, 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')
    else:
        # Append the w50, its error and the peak flux to the main list
        xmmlss_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        xmmlss14_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_w50_flux_z_D_himass_vsys.append([2*w50_data[1][3], 2*w50_data[2][3], max(prof_data[1][2:]), z_data[1][i], z_data[3][i], xmmlss14_1380_dets[i]['MHI(1e9)']*1e9, himass_data[3][i]])
        mightee_eff.append('')

In [222]:
print(len(mightee_eff), len(mightee_w50_flux_z_D_himass_vsys)+len(mightee_w50_flux_z_D_himass_vsys_excl))
print(len(cosm_w50_flux_z_D_himass_vsys)+len(cosm_w50_flux_z_D_himass_vsys_excl)+len(xmmlss_w50_flux_z_D_himass_vsys)+len(xmmlss_w50_flux_z_D_himass_vsys_excl))

267 267
267


## Velocity function using number density

For a simple idea of the velocity function, we will first calculate the velocity function in the form of a number density. To do that, we first have to calculate the total volume in which the detections lie. 

The detections found were in the XMMLSS and COSMOS fields. We can use the ```astropy.cosmology``` package to calculate the maximum comoving volume of the sky in Mpc$^3$ from $z = 0$ to $z_{max} = 0.084791$, which is the redshift of the lowest frequency ($1310$ MHz) of the MIGHTEE observations of those fields. And then multiply that by the fractional area each of the fields are.

In [223]:
# Cosmology parameters based on MIGHTEE
H0 = 67.4 # km/s/Mpc
h = H0/100
Om0 = 0.315
Ode0 = 0.685

cosmo = LambdaCDM(H0, Om0, Ode0)

In [224]:
zmax_mightee = 0.084791 # Redshift at 1310 MHz

# Comoving volume of the whole sky
sky_cos_vol = cosmo.comoving_volume(zmax_mightee)*pow(h, 3)
sky_cos_vol

<Quantity 64691516.72187506 Mpc3>

Generally, majority of the detections lie within a $0.7^{\circ} $ radius (as opposed to the full $1^{\circ} $ radius). And so the effective area can be estimated as $\pi (0.7)^2 = 1.539$ deg$^2$. For XMMLSS, we have to be more cautious since the pointing overlap. 

![XMM_HI_detections.pdf](XMM_HI_detections.pdf)

Note that ```cosmo.comoving_volume``` calculates the comoving volume for the full sphere up to the redshift that is parsed in. So you have to multiply by the fractional area so you get the fractional comoving volume.

In [231]:
# Chose effective area of 1.5 deg2
# This means HPBW = 0.7

# This link is also useful for checking the function below 
#http://www.ambrsoft.com/TrigoCalc/Circles2/circle2intersection/CircleCircleIntersection.htm
# Second using function below
def areaOfIntersection(x0, y0, r0, x1, y1, r1):
    rr0 = r0*r0
    rr1 = r1*r1
    c = math.sqrt((x1-x0)*(x1-x0) + (y1-y0)*(y1-y0))
    phi = (math.acos((rr0+(c*c)-rr1) / (2*r0*c)))*2
    theta = (math.acos((rr1+(c*c)-rr0) / (2*r1*c)))*2
    area1 = 0.5*theta*rr1 - 0.5*rr1*math.sin(theta)
    area2 = 0.5*phi*rr0 - 0.5*rr0*math.sin(phi)
    return area1 + area2

# Average of two methods
xmmlss_crescent = areaOfIntersection(0, 0, 0.7, 0.7, 0, 0.7)*pow(u.deg,2) # x0 = y0 = 0, x1=0.7, y1=0

In [238]:
# Areas of fields
cosmos_area = 1.539*pow(u.deg,2)
xmmlss12_area = 1.539*pow(u.deg,2)
xmmlss13_area = 1.539*pow(u.deg,2)
xmmlss14_area = 1.539*pow(u.deg,2)
xmmlss_area = (xmmlss12_area - xmmlss_crescent) + xmmlss13_area + (xmmlss14_area - xmmlss_crescent)
mightee_area = cosmos_area + xmmlss_area

# Area of full sky
sky_area = 41253*pow(u.deg,2)

# Fractional areas
mightee_frac_area = mightee_area/sky_area
cosmos_frac_area = cosmos_area/sky_area
xmmlss_frac_area = xmmlss_area/sky_area
xmmlss12_frac_area = xmmlss12_area/sky_area
xmmlss13_frac_area = xmmlss13_area/sky_area
xmmlss14_frac_area = xmmlss14_area/sky_area

In [241]:
# Comoving volumes of COSMOS and XMMLSS fields
mightee_vol = sky_cos_vol*mightee_frac_area
cosmos_vol = sky_cos_vol*cosmos_frac_area
xmmlss_vol = sky_cos_vol*xmmlss_frac_area
xmmlss12_vol = sky_cos_vol*xmmlss12_frac_area
xmmlss13_vol = sky_cos_vol*xmmlss13_frac_area
xmmlss14_vol = sky_cos_vol*xmmlss14_frac_area

### Number Density: W50
Now that we have the total comoving volume of the fields, we need to calculate a number density for each detection.

In [245]:
# Extract w50 values
mightee_w50 = [np.log10(w50[0]) for w50 in mightee_w50_flux_z_D_himass_vsys]
cosm_w50 = [np.log10(w50[0]) for w50 in cosm_w50_flux_z_D_himass_vsys]
xmmlss_w50 = [np.log10(w50[0]) for w50 in xmmlss_w50_flux_z_D_himass_vsys]
xmmlss12_w50 = [np.log10(w50[0]) for w50 in xmmlss12_w50_flux_z_D_himass_vsys]
xmmlss13_w50 = [np.log10(w50[0]) for w50 in xmmlss13_w50_flux_z_D_himass_vsys]
xmmlss14_w50 = [np.log10(w50[0]) for w50 in xmmlss14_w50_flux_z_D_himass_vsys]

In [246]:
print('MIGHTEE:', len(mightee_w50))
print('COSMOS:', len(cosm_w50))
print('XMMLSS12:', len(xmmlss12_w50))
print('XMMLSS13:', len(xmmlss13_w50))
print('XMMLSS14:', len(xmmlss14_w50))

MIGHTEE: 232
COSMOS: 60
XMMLSS12: 53
XMMLSS13: 56
XMMLSS14: 63


In [247]:
print('Minimum W50 =', pow(10, min(mightee_w50)), 'km/s')

Minimum W50 = 44.10000000000001 km/s


In [248]:
print('Maximum W50 =', pow(10, max(mightee_w50)), 'km/s')

Maximum W50 = 505.9825508592083 km/s


In [249]:
# Bins for the histogram
w50_min = np.log10(44)
w50_max = np.log10(528)

bin_width = 0.09726890629795543 # Size of W50 bins from Velocity_function.ipynb
#no_bins = (w50_max - w50_min)/bin_width

#w50_bins = np.arange(w50_min, w50_max + bin_width, bin_width)
w50_bins = np.array([1.94448267-3*bin_width, 1.94448267-2*bin_width, 1.94448267-1*bin_width, 
                     1.94448267, 2.04175158, 2.13902048, 2.23628939, 2.3335583,
                       2.4308272, 2.52809611, 2.62536502, 2.72263392]) # this the bottom two lines of this array are from Velocity_function.ipynb
no_bins = len(w50_bins)

In [250]:
# Necessary functions for plotting
def tick_function(x):
    '''
    Returns the tick format of twin x axis
    '''
    return ["%s" % i for i in x]

def func(x):
    '''
    Returns the logarithm 
    '''
    funcx = np.log10(x)
    return funcx

In [251]:
# Create a directory to store all the plots

# Path for new directory that will be created
dirName = ''

# if not os.path.exists(dirName):
#     os.makedirs(dirName)
#     print("Directory", dirName, "created.")
# else:    
#     print("Directory", dirName, "already exists and will be overwritten.")
#     shutil.rmtree(dirName)
#     os.makedirs(dirName)

In [ ]:
# Simple W50 number density plot
'''MIGHTEE'''
mightee_counts, mightee_bin_edges, _ = pl.hist(mightee_w50, bins=w50_bins, color='white')
mightee_bin_centres = (mightee_bin_edges[:-1] + mightee_bin_edges[1:])/2. 

'''COSMOS'''
cosm_counts, cosm_bin_edges, _ = pl.hist(cosm_w50, bins=w50_bins, color='white')
cosm_bin_centres = (cosm_bin_edges[:-1] + cosm_bin_edges[1:])/2.

'''XMMLSS'''
xmmlss_counts, xmmlss_bin_edges, _ = pl.hist(xmmlss_w50, bins=w50_bins, color='white')
xmmlss_bin_centres = (xmmlss_bin_edges[:-1] + xmmlss_bin_edges[1:])/2.

xmmlss12_counts, xmmlss12_bin_edges, _ = pl.hist(xmmlss12_w50, bins=w50_bins, color='white')
xmmlss12_bin_centres = (xmmlss12_bin_edges[:-1] + xmmlss12_bin_edges[1:])/2.

xmmlss13_counts, xmmlss13_bin_edges, _ = pl.hist(xmmlss13_w50, bins=w50_bins, color='white')
xmmlss13_bin_centres = (xmmlss13_bin_edges[:-1] + xmmlss13_bin_edges[1:])/2.

xmmlss14_counts, xmmlss14_bin_edges, _ = pl.hist(xmmlss14_w50, bins=w50_bins, color='white')
xmmlss14_bin_centres = (xmmlss14_bin_edges[:-1] + xmmlss14_bin_edges[1:])/2.

# Definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom - 0.2, width, 0.2]

# start with a rectangular Figure
plt.figure(figsize=(15, 10))

# Scatter plot axes set up
ax_scatter = plt.axes(rect_scatter)
ax_scatter.xaxis.set_minor_locator(AutoMinorLocator())
ax_scatter.yaxis.set_minor_locator(AutoMinorLocator())
ax_scatter.axes.tick_params(axis='both', which='minor', length=5, direction='in', labelsize=24, top=True, right=True)
ax_scatter.axes.tick_params(axis='both', which='major', length=10, direction='in', labelsize=24, top=True, right=True)
ax_scatter.axes.tick_params(axis='x', pad=20)
ax_scatter.tick_params(axis='both', labelsize=24, labeltop=False, labelbottom=False)
ax_scatter.xaxis.set_label_position('top')
ax_scatter.set_xlabel(r'$W_{50}$ [km s$^{-1}$]', fontsize=24, labelpad=40)
ax_scatter.set_ylabel(r'log ($n$/h$^3$Mpc$^{-3}$)', fontsize=24)

# Number of galaxies vs log w
ax_histx = plt.axes(rect_histx)
ax_histx.xaxis.set_minor_locator(AutoMinorLocator())
ax_histx.yaxis.set_minor_locator(AutoMinorLocator())
ax_histx.axes.tick_params(axis='x', which='minor', length=5, direction='in', labelsize=24, top=True, bottom=True)
ax_histx.axes.tick_params(axis='x', which='major', length=10, direction='in', labelsize=24, top=True, bottom=True)
ax_histx.axes.tick_params(axis='y', which='minor', length=5, direction='in', labelsize=24, right=True)
ax_histx.axes.tick_params(axis='y', which='major', length=10, direction='in', labelsize=24, right=True)
ax_histx.tick_params(direction='in', labelbottom=True, labeltop=False)
ax_histx.xaxis.set_label_position('bottom')
ax_histx.set_xlabel(r'log ($W_{50}$/km s$^{-1}$)', fontsize=24, labelpad=15)
ax_histx.set_ylabel(r'$N_\mathrm{gal}$', fontsize=24)
ax_histx.tick_params(axis='both', labelsize=24)

'''MIGHTEE'''
ax_scatter.plot(mightee_bin_centres[3:], np.log10(mightee_counts/mightee_vol.value)[3:], 'o', color='black', markersize=12, label='MIGHTEE-HI', alpha=0.8)
ax_scatter.plot(mightee_bin_centres[0:3], np.log10(mightee_counts/mightee_vol.value)[0:3], 'o', color='black', markerfacecolor='white', markersize=12, label=None, alpha=0.8)
    
''' COSMOS'''
ax_scatter.plot(cosm_bin_centres[3:], np.log10(cosm_counts/cosmos_vol.value)[3:], 'o', color='royalblue', markersize=12, label='COSMOS', alpha=0.8)
ax_scatter.plot(cosm_bin_centres[0:3], np.log10(cosm_counts/cosmos_vol.value)[0:3], 'o', color='royalblue', markerfacecolor='white', markersize=12, label=None, alpha=0.8)
    
'''XMMLSS'''
ax_scatter.plot(xmmlss_bin_centres[3:], np.log10(xmmlss_counts/xmmlss_vol.value)[3:], 'o', color='red', markersize=12, label='XMM-LSS', alpha=0.8)
ax_scatter.plot(xmmlss_bin_centres[0:3], np.log10(xmmlss_counts/xmmlss_vol.value)[0:3], 'o', color='red', markerfacecolor='white', markersize=12, label=None, alpha=0.8)

# '''XMMLSS12'''
# ax_scatter.plot(xmmlss12_bin_centres, np.log10(xmmlss12_counts/xmmlss12_vol.value), 's', color='salmon', markersize=8, label='XMM-LSS 12', alpha=0.9)
    
# '''XMMLSS13'''
# ax_scatter.plot(xmmlss13_bin_centres, np.log10(xmmlss13_counts/xmmlss13_vol.value), 's', color='chocolate', markersize=8, label='XMM-LSS 13', alpha=0.9)
    
# '''XMMLSS14'''
# ax_scatter.plot(xmmlss14_bin_centres, np.log10(xmmlss14_counts/xmmlss14_vol.value), 's', color='maroon', markersize=8, label='XMM-LSS 14', alpha=0.9)

# Limits
# Limits
ax_scatter.set_xlim((w50_min-0.14, w50_max+0.14))
ax_scatter.set_ylim((-3.9, -1.9))

# Histograms
ax_histx.hist([mightee_w50, cosm_w50, xmmlss_w50], bins=w50_bins, stacked=False, rwidth=0.9, edgecolor='black', color=['black', 'royalblue', 'red'])

# Label each bin
for rect in ax_histx.patches:
    height = rect.get_height()
    ax_histx.annotate(f'{int(height)}', xy=(rect.get_x()+rect.get_width()/2, height), 
                xytext=(0.5, 5), textcoords='offset points', ha='center', va='bottom', fontsize=12.2) 

# Tick positions of the twin x axis
ax_histx.set_ylim((0, 64))
ax_histx.set_xlim(ax_scatter.get_xlim())

# Tick positions of the twin x axis
# Set up the second axis
ax_scatter2 = ax_scatter.twiny()
ax_scatter2.axes.tick_params(axis='both', which = 'major', length=10, direction='in')
ax_scatter2.axes.tick_params(axis='x', which = 'minor', length=5, direction='in')

zticks = np.array([30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900])
zticks_labels = ['', '40', '', '60', '', '80', '', '100', '200', '300', '400', '500', '', '700', '800', '900']
ax_scatter2.set_xticks(func(zticks))
ax_scatter2.set_xticklabels(tick_function(zticks_labels))
ax_scatter_bounds = ax_scatter.get_xbound()
ax_scatter2.set_xlim(ax_scatter_bounds[0], ax_scatter_bounds[1])
ax_scatter2.axes.tick_params(labelsize=24)

# Legend
ax_scatter.legend(loc='lower left', fontsize=24, frameon=False)

pl.savefig(dirName+'no_density.pdf', bbox_inches='tight', dpi=300)

plt.show()

## Velocity function using 1/V$_\mathrm{max}$ method (comoving volumes, Zwaan 2010 version)

In [254]:
# Limiting peak flux of MIGHTEE
# Values attained from the MeerKAT Specs page for 1350 MHz http://public.ska.ac.za/meerkat/meerkat-schedule
SEFD = 443 # System equivalent integrated flux in Jy
Tsys_eta = 23 # Tsys divided by the system efficiency in K
Ae_Tsys = 6.22 # Effective area divided by Tsys in m^2/K

# Variable for calculating the theoretical sensitivity in Jy
N = 62 # Number of antennas
eta = 1 # System efficiency
deltat = 6.5*60*60 # Integration time of 6h in seconds
deltanu = 208985.82 # Frequency width in Hz

# Calculate sigma in Jy
sigma = 1.4*(SEFD/(eta*np.sqrt(2*N*(N-1)*deltat*deltanu)))
print("Sigma = %f uJy"%(sigma*u.Jy.to(u.uJy)))

Sigma = 101.974049 uJy


The HI mass is calculated using

\begin{equation}
\left(\frac{M_\text{HI}}{\text{M}_\odot}\right) \approx {2.36 \times 10^5} \left(\frac{D_L}{\text{Mpc}}\right)^2 \int\left[\frac{S(\nu)}{\text{Jy}}\right]\left(\frac{dv}{\text{km/s}}\right) \\
\end{equation}

For each detection, we need to calculate $Sdv$ over a range of redshifts. The first instance $Sdv <= Sdv_{lim}$ will then be used to calculate the redshift $z_{max}$. $Sdv_{lim} = 5(W_{50}/44)\sigma$

\begin{align}
Sdv = \frac{M_\text{HI}}{(2.356 \times 10^5)D_L^2}
\end{align}

The luminosity distance $D_L$ is also a function of redshift given by

\begin{align}
D_L = (1+z)D_C,
\end{align}

where $D_C$ is the comoving distance given by

\begin{align}
D_C = \frac{c}{H_0}\int^z_0 \frac{dz'}{\sqrt{\Omega_m(1+z)^3 + \Omega_\Lambda}}
\end{align}

For MIGHTEE, the cosmological constants used to initialise ```cosmo``` are $H_0$, $\Omega_m$ and $\Omega_\Lambda$, and that is why $E(z)$ reduces to what it is.

In [256]:
# Redshift ranges
z_mightee = np.arange(0, zmax_mightee, 0.0005)

In [257]:
'''MIGHTEE'''
# List to store Vmax and zmax values
mightee_Vmax_list = []
mightee_zmax_list = []

for i in range(len(mightee_w50_flux_z_D_himass_vsys)):
    # Extract HI mass and distance for detection i
    w50_det = mightee_w50_flux_z_D_himass_vsys[i][0]
    Mhi = mightee_w50_flux_z_D_himass_vsys[i][5]
     
    # Line width sigma
    sigma_det = 5*np.sqrt((w50_det/44)*sigma)
    
    # Buffer for zmax if Sdv is never <= Sdvlim
    zmax = 'null'

    # Loop through the redshifts in z_mightee
    for z in z_mightee:
        # Comoving luminosity distance
        DL = (cosmo.comoving_distance(z).value)*(1 + z)
        
        # Calculate Sdv
        Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))

        # Check if Sdv <= sigma_det
        if Sdv <= sigma_det:
            # If true, select z as zmax
            zmax = z
            mightee_zmax_list.append(z)
            break
    
    # If none of the redshifts resulted in Sdv <= Sdvlim
    # Set zmax to zmax_mightee
    if zmax == 'null':
        mightee_zmax_list.append(zmax_mightee)
    
    # Calculate Vmax
    mightee_Vmax = (mightee_frac_area)*cosmo.comoving_volume(mightee_zmax_list[i])*pow(h, 3)
    mightee_Vmax_list.append(mightee_Vmax.value)

<ipython-input-257-4223bbebff91>:23: RuntimeWarning: divide by zero encountered in double_scalars
  Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))


In [258]:
'''COSMOS'''
# List to store Vmax and zmax values
cosmos_Vmax_list = []
cosmos_zmax_list = []

for i in range(len(cosm_w50_flux_z_D_himass_vsys)):
    # Extract HI mass and distance for detection i
    w50_det = cosm_w50_flux_z_D_himass_vsys[i][0]
    Mhi = cosm_w50_flux_z_D_himass_vsys[i][5]
     
    # Line width sigma
    sigma_det = 5*np.sqrt((w50_det/44)*sigma)
    
    # Buffer for zmax if Sdv is never <= Sdvlim
    zmax = 'null'

    # Loop through the redshifts in z_mightee
    for z in z_mightee:
        # Comoving luminosity distance
        DL = (cosmo.comoving_distance(z).value)*(1 + z)
        
        # Calculate Sdv
        Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))

        # Check if Sdv <= sigma_det
        if Sdv <= sigma_det:
            # If true, select z as zmax
            zmax = z
            cosmos_zmax_list.append(z)
            break
    
    # If none of the redshifts resulted in Sdv <= Sdvlim
    # Set zmax to zmax_mightee
    if zmax == 'null':
        cosmos_zmax_list.append(zmax_mightee)
    
    # Calculate Vmax
    cosmos_Vmax = (cosmos_frac_area)*cosmo.comoving_volume(cosmos_zmax_list[i])*pow(h, 3)
    cosmos_Vmax_list.append(cosmos_Vmax.value)

<ipython-input-258-8e1722adf2cc>:23: RuntimeWarning: divide by zero encountered in double_scalars
  Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))


In [259]:
'''XMMLSS'''
# List to store Vmax and zmax values
xmmlss_Vmax_list = []
xmmlss_zmax_list = []

for i in range(len(xmmlss_w50_flux_z_D_himass_vsys)):
    # Extract w50, HI mass and distance for detection i
    w50_det = xmmlss_w50_flux_z_D_himass_vsys[i][0]
    Mhi = xmmlss_w50_flux_z_D_himass_vsys[i][5]
    
    # Line width sigma
    sigma_det = 5*np.sqrt((w50_det/44)*sigma)
    
    # Buffer for zmax if Sdv is never <= Sdvlim
    zmax = 'null'

    # Loop through the redshifts in z_mightee
    for z in z_mightee:
        # Comoving luminosity distance
        DL = (cosmo.comoving_distance(z).value)*(1 + z)
        
        # Calculate Sdv
        Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))

        # Check if Sdv <= sigma_det
        if Sdv <= sigma_det:
            # If true, select z as zmax
            zmax = z
            xmmlss_zmax_list.append(z)
            break
    
    # If none of the redshifts resulted in Sdv <= Sdvlim
    # Set zmax to zmax_mightee
    if zmax == 'null':
        xmmlss_zmax_list.append(zmax_mightee)
    
    # Calculate Vmax
    xmmlss_Vmax = (xmmlss_frac_area)*cosmo.comoving_volume(xmmlss_zmax_list[i])*pow(h, 3)
    xmmlss_Vmax_list.append(xmmlss_Vmax.value)

<ipython-input-259-1dd426b3b023>:23: RuntimeWarning: divide by zero encountered in double_scalars
  Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))


In [260]:
'''XMMLSS12'''
# List to store Vmax and zmax values
xmmlss12_Vmax_list = []
xmmlss12_zmax_list = []

for i in range(len(xmmlss12_w50_flux_z_D_himass_vsys)):
    # Extract w50, HI mass and distance for detection i
    w50_det = xmmlss12_w50_flux_z_D_himass_vsys[i][0]
    Mhi = xmmlss12_w50_flux_z_D_himass_vsys[i][5]
    
    # Line width sigma
    sigma_det = 5*np.sqrt((w50_det/44)*sigma)
    
    # Buffer for zmax if Sdv is never <= Sdvlim
    zmax = 'null'

    # Loop through the redshifts in z_mightee
    for z in z_mightee:
        # Comoving luminosity distance
        DL = (cosmo.comoving_distance(z).value)*(1 + z)
        
        # Calculate Sdv
        Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))

        # Check if Sdv <= sigma_det
        if Sdv <= sigma_det:
            # If true, select z as zmax
            zmax = z
            xmmlss12_zmax_list.append(z)
            break
    
    # If none of the redshifts resulted in Sdv <= Sdvlim
    # Set zmax to zmax_mightee
    if zmax == 'null':
        xmmlss12_zmax_list.append(zmax_mightee)
    
    # Calculate Vmax
    xmmlss12_Vmax = (xmmlss12_frac_area)*cosmo.comoving_volume(xmmlss12_zmax_list[i])*pow(h, 3)
    xmmlss12_Vmax_list.append(xmmlss12_Vmax.value)

<ipython-input-260-6cf64d6513cb>:23: RuntimeWarning: divide by zero encountered in double_scalars
  Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))


In [261]:
'''XMMLSS13'''
# List to store Vmax and zmax values
xmmlss13_Vmax_list = []
xmmlss13_zmax_list = []

for i in range(len(xmmlss13_w50_flux_z_D_himass_vsys)):
    # Extract w50, HI mass and distance for detection i
    w50_det = xmmlss13_w50_flux_z_D_himass_vsys[i][0]
    Mhi = xmmlss13_w50_flux_z_D_himass_vsys[i][5]
    
    # Line width sigma
    sigma_det = 5*np.sqrt((w50_det/44)*sigma)
    
    # Buffer for zmax if Sdv is never <= Sdvlim
    zmax = 'null'

    # Loop through the redshifts in z_mightee
    for z in z_mightee:
        # Comoving luminosity distance
        DL = (cosmo.comoving_distance(z).value)*(1 + z)
        
        # Calculate Sdv
        Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))

        # Check if Sdv <= sigma_det
        if Sdv <= sigma_det:
            # If true, select z as zmax
            zmax = z
            xmmlss13_zmax_list.append(z)
            break
    
    # If none of the redshifts resulted in Sdv <= Sdvlim
    # Set zmax to zmax_mightee
    if zmax == 'null':
        xmmlss13_zmax_list.append(zmax_mightee)
    
    # Calculate Vmax
    xmmlss13_Vmax = (xmmlss13_frac_area)*cosmo.comoving_volume(xmmlss13_zmax_list[i])*pow(h, 3)
    xmmlss13_Vmax_list.append(xmmlss13_Vmax.value)

<ipython-input-261-e338e87af0ff>:23: RuntimeWarning: divide by zero encountered in double_scalars
  Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))


In [262]:
'''XMMLSS14'''
# List to store Vmax and zmax values
xmmlss14_Vmax_list = []
xmmlss14_zmax_list = []

for i in range(len(xmmlss14_w50_flux_z_D_himass_vsys)):
    # Extract w50, HI mass and distance for detection i
    w50_det = xmmlss14_w50_flux_z_D_himass_vsys[i][0]
    Mhi = xmmlss14_w50_flux_z_D_himass_vsys[i][5]
    
    # Line width sigma
    sigma_det = 5*np.sqrt((w50_det/44)*sigma)
    
    # Buffer for zmax if Sdv is never <= Sdvlim
    zmax = 'null'

    # Loop through the redshifts in z_mightee
    for z in z_mightee:
        # Comoving luminosity distance
        DL = (cosmo.comoving_distance(z).value)*(1 + z)
        
        # Calculate Sdv
        Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))

        # Check if Smin <= sigma_det
        if Sdv <= sigma_det:
            # If true, select z as zmax
            zmax = z
            xmmlss14_zmax_list.append(z)
            break
    
    # If none of the redshifts resulted in Smin <= Sdvlim
    # Set zmax to zmax_mightee
    if zmax == 'null':
        xmmlss14_zmax_list.append(zmax_mightee)
    
    # Calculate Vmax
    xmmlss14_Vmax = (xmmlss14_frac_area)*cosmo.comoving_volume(xmmlss14_zmax_list[i])*pow(h, 3)
    xmmlss14_Vmax_list.append(xmmlss14_Vmax.value)

<ipython-input-262-99ee32bdc2dd>:23: RuntimeWarning: divide by zero encountered in double_scalars
  Sdv = (Mhi*pow(1, 2))/(2.356*1e5*pow(DL, 2))


The next step is to construct the velocity function. We need to create bins of $W_{50}$.

In [263]:
# Need to get the bin IDs for each detection
mightee_bin_id = np.digitize(mightee_w50, w50_bins)

cosm_bin_id = np.digitize(cosm_w50, w50_bins)

xmmlss_bin_id = np.digitize(xmmlss_w50, w50_bins)
xmmlss12_bin_id = np.digitize(xmmlss12_w50, w50_bins)
xmmlss13_bin_id = np.digitize(xmmlss13_w50, w50_bins)
xmmlss14_bin_id = np.digitize(xmmlss14_w50, w50_bins)

In [264]:
uniquea, countsa = np.unique(mightee_bin_id, return_counts=True)
print('Number of detections in each bin')
dict(zip(uniquea, countsa))

Number of detections in each bin


{0: 7,
 1: 17,
 2: 27,
 3: 14,
 4: 43,
 5: 24,
 6: 19,
 7: 33,
 8: 27,
 9: 15,
 10: 4,
 11: 2}

In [265]:
# Lists to store scatter plot data for HI mass vs Vsys and Line width vs redshift for each bin
vsys1 = []
himass1 = []
width1 = []
redshift1 = []

vsys2 = []
himass2 = []
width2 = []
redshift2 = []

vsys3 = []
himass3 = []
width3 = []
redshift3 = []

vsys4 = []
himass4 = []
width4 = []
redshift4 = []

vsys5 = []
himass5 = []
width5 = []
redshift5 = []

vsys6 = []
himass6 = []
width6 = []
redshift6 = []

vsys7 = []
himass7 = []
width7 = []
redshift7 = []

vsys8 = []
himass8 = []
width8 = []
redshift8 = []

vsys9 = []
himass9 = []
width9 = []
redshift9 = []

vsys10 = []
himass10 = []
width10 = []
redshift10 = []

vsys11 = []
himass11 = []
width11 = []
redshift11 = []

In [266]:
'''MIGHTEE'''
# List to store 1/Vmax bins for W50s
mightee_phi_Vmax = []
mightee_phi_Vmax_err = []

mightee_bin_id_count = 1 # Counter for bin ID

while mightee_bin_id_count < len(w50_bins):
    # Sum for bin
    mightee_sum_bin_Vmax = 0
    mightee_sum_bin_Vmax_err = 0
    
    for i in range(len(mightee_bin_id)):
        if mightee_bin_id[i] == mightee_bin_id_count:
            mightee_sum_bin_Vmax += 1/mightee_Vmax_list[i]
            mightee_sum_bin_Vmax_err += pow(1/mightee_Vmax_list[i], 2)
            
            # For the scatter plots
            if mightee_bin_id[i] == 1:
                vsys1.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass1.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width1.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift1.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 2:
                vsys2.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass2.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width2.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift2.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 3:
                vsys3.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass3.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width3.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift3.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 4:
                vsys4.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass4.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width4.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift4.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 5:
                vsys5.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass5.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width5.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift5.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 6:
                vsys6.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass6.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width6.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift6.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 7:
                vsys7.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass7.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width7.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift7.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 8:
                vsys8.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass8.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width8.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift8.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 9:
                vsys9.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass9.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width9.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift9.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 10:
                vsys10.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass10.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width10.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift10.append(mightee_w50_flux_z_D_himass_vsys[i][3])
            elif mightee_bin_id[i] == 11:
                vsys11.append(mightee_w50_flux_z_D_himass_vsys[i][6])
                himass11.append(np.log10(mightee_w50_flux_z_D_himass_vsys[i][5]))
                width11.append(mightee_w50_flux_z_D_himass_vsys[i][0])
                redshift11.append(mightee_w50_flux_z_D_himass_vsys[i][3])

    mightee_phi_Vmax.append(mightee_sum_bin_Vmax)
    mightee_phi_Vmax_err.append(np.sqrt(mightee_sum_bin_Vmax_err))
    
    # Increment bin ID counter
    mightee_bin_id_count += 1
    
mightee_phi_Vmax_upper_err = np.array(mightee_phi_Vmax) + np.array(mightee_phi_Vmax_err)
mightee_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(mightee_phi_Vmax))) - np.abs(np.log10(mightee_phi_Vmax_upper_err)))

mightee_phi_Vmax_lower_err = np.array(mightee_phi_Vmax) - np.array(mightee_phi_Vmax_err)
mightee_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(mightee_phi_Vmax))) - np.abs(np.log10(mightee_phi_Vmax_lower_err)))
mightee_phi_Vmax_lower_err_size_log = np.where(mightee_phi_Vmax_lower_err_size_log == float('inf'), 20,
                                            mightee_phi_Vmax_lower_err_size_log)

In [267]:
'''COSMOS'''
# List to store 1/Vmax bins for W50s
cosm_phi_Vmax = []
cosm_phi_Vmax_err = []

cosm_bin_id_count = 1 # Counter for bin ID

while cosm_bin_id_count < len(w50_bins):
    # Sum for bin
    cosm_sum_bin_Vmax = 0
    cosm_sum_bin_Vmax_err = 0
    
    for i in range(len(cosm_bin_id)):
        if cosm_bin_id[i] == cosm_bin_id_count:
            cosm_sum_bin_Vmax += 1/cosmos_Vmax_list[i]
            cosm_sum_bin_Vmax_err += pow(1/cosmos_Vmax_list[i], 2)

    cosm_phi_Vmax.append(cosm_sum_bin_Vmax)
    cosm_phi_Vmax_err.append(np.sqrt(cosm_sum_bin_Vmax_err))
    
    # Increment bin ID counter
    cosm_bin_id_count += 1
    
cosm_phi_Vmax_upper_err = np.array(cosm_phi_Vmax) + np.array(cosm_phi_Vmax_err)
cosm_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(cosm_phi_Vmax))) - np.abs(np.log10(cosm_phi_Vmax_upper_err)))

cosm_phi_Vmax_lower_err = np.array(cosm_phi_Vmax) - np.array(cosm_phi_Vmax_err)
cosm_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(cosm_phi_Vmax))) - np.abs(np.log10(cosm_phi_Vmax_lower_err)))
cosm_phi_Vmax_lower_err_size_log = np.where(cosm_phi_Vmax_lower_err_size_log == float('inf'), 20,
                                            cosm_phi_Vmax_lower_err_size_log)

<ipython-input-267-325031b81279>:25: RuntimeWarning: divide by zero encountered in log10
  cosm_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(cosm_phi_Vmax))) - np.abs(np.log10(cosm_phi_Vmax_upper_err)))
<ipython-input-267-325031b81279>:25: RuntimeWarning: invalid value encountered in subtract
  cosm_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(cosm_phi_Vmax))) - np.abs(np.log10(cosm_phi_Vmax_upper_err)))
<ipython-input-267-325031b81279>:28: RuntimeWarning: divide by zero encountered in log10
  cosm_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(cosm_phi_Vmax))) - np.abs(np.log10(cosm_phi_Vmax_lower_err)))
<ipython-input-267-325031b81279>:28: RuntimeWarning: invalid value encountered in subtract
  cosm_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(cosm_phi_Vmax))) - np.abs(np.log10(cosm_phi_Vmax_lower_err)))


In [268]:
'''XMMLSS'''
# List to store 1/Vmax bins for W50s
xmmlss_phi_Vmax = []
xmmlss_phi_Vmax_err = []

xmmlss_bin_id_count = 1 # Counter for bin ID

while xmmlss_bin_id_count < len(w50_bins):
    # Sum for bin 
    xmmlss_sum_bin_Vmax = 0
    xmmlss_sum_bin_Vmax_err = 0
    
    for i in range(len(xmmlss_bin_id)):
        if xmmlss_bin_id[i] == xmmlss_bin_id_count:
            xmmlss_sum_bin_Vmax += 1/xmmlss_Vmax_list[i]
            xmmlss_sum_bin_Vmax_err += pow(1/xmmlss_Vmax_list[i], 2)
            
    xmmlss_phi_Vmax.append(xmmlss_sum_bin_Vmax)
    xmmlss_phi_Vmax_err.append(np.sqrt(xmmlss_sum_bin_Vmax_err))
    
    # Increment bin ID counter
    xmmlss_bin_id_count += 1
    
xmmlss_phi_Vmax_upper_err = np.array(xmmlss_phi_Vmax) + np.array(xmmlss_phi_Vmax_err)
xmmlss_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss_phi_Vmax))) - np.abs(np.log10(xmmlss_phi_Vmax_upper_err)))

xmmlss_phi_Vmax_lower_err = np.array(xmmlss_phi_Vmax) - np.array(xmmlss_phi_Vmax_err)
xmmlss_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss_phi_Vmax))) - np.abs(np.log10(xmmlss_phi_Vmax_lower_err)))

In [269]:
'''XMMLSS12'''
# List to store 1/Vmax bins for W50s
xmmlss12_phi_Vmax = []
xmmlss12_phi_Vmax_err = []

xmmlss12_bin_id_count = 1 # Counter for bin ID

while xmmlss12_bin_id_count < len(w50_bins):
    # Sum for bin 
    xmmlss12_sum_bin_Vmax = 0
    xmmlss12_sum_bin_Vmax_err = 0
    
    for i in range(len(xmmlss12_bin_id)):
        if xmmlss12_bin_id[i] == xmmlss12_bin_id_count:
            xmmlss12_sum_bin_Vmax += 1/xmmlss12_Vmax_list[i]
            xmmlss12_sum_bin_Vmax_err += pow(1/xmmlss12_Vmax_list[i], 2)

    xmmlss12_phi_Vmax.append(xmmlss12_sum_bin_Vmax)
    xmmlss12_phi_Vmax_err.append(np.sqrt(xmmlss12_sum_bin_Vmax_err))
    
    # Increment bin ID counter
    xmmlss12_bin_id_count += 1
    
xmmlss12_phi_Vmax_upper_err = np.array(xmmlss12_phi_Vmax) + np.array(xmmlss12_phi_Vmax_err)
xmmlss12_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss12_phi_Vmax))) - np.abs(np.log10(xmmlss12_phi_Vmax_upper_err)))

xmmlss12_phi_Vmax_lower_err = np.array(xmmlss12_phi_Vmax) - np.array(xmmlss12_phi_Vmax_err)
xmmlss12_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss12_phi_Vmax))) - np.abs(np.log10(xmmlss12_phi_Vmax_lower_err)))
xmmlss12_phi_Vmax_lower_err_size_log = np.where(xmmlss12_phi_Vmax_lower_err_size_log == float('inf'), 20,
                                            xmmlss12_phi_Vmax_lower_err_size_log)

<ipython-input-269-d5838f8e2795>:25: RuntimeWarning: divide by zero encountered in log10
  xmmlss12_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss12_phi_Vmax))) - np.abs(np.log10(xmmlss12_phi_Vmax_upper_err)))
<ipython-input-269-d5838f8e2795>:25: RuntimeWarning: invalid value encountered in subtract
  xmmlss12_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss12_phi_Vmax))) - np.abs(np.log10(xmmlss12_phi_Vmax_upper_err)))
<ipython-input-269-d5838f8e2795>:28: RuntimeWarning: divide by zero encountered in log10
  xmmlss12_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss12_phi_Vmax))) - np.abs(np.log10(xmmlss12_phi_Vmax_lower_err)))
<ipython-input-269-d5838f8e2795>:28: RuntimeWarning: invalid value encountered in subtract
  xmmlss12_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss12_phi_Vmax))) - np.abs(np.log10(xmmlss12_phi_Vmax_lower_err)))


In [270]:
'''XMMLSS13'''
# List to store 1/Vmax bins for W50s
xmmlss13_phi_Vmax = []
xmmlss13_phi_Vmax_err = []

xmmlss13_bin_id_count = 1 # Counter for bin ID

while xmmlss13_bin_id_count < len(w50_bins):
    # Sum for bin 
    xmmlss13_sum_bin_Vmax = 0
    xmmlss13_sum_bin_Vmax_err = 0
    
    for i in range(len(xmmlss13_bin_id)):
        if xmmlss13_bin_id[i] == xmmlss13_bin_id_count:
            xmmlss13_sum_bin_Vmax += 1/xmmlss13_Vmax_list[i]
            xmmlss13_sum_bin_Vmax_err += pow(1/xmmlss13_Vmax_list[i], 2)

    xmmlss13_phi_Vmax.append(xmmlss13_sum_bin_Vmax)
    xmmlss13_phi_Vmax_err.append(np.sqrt(xmmlss13_sum_bin_Vmax_err))
    
    # Increment bin ID counter
    xmmlss13_bin_id_count += 1

xmmlss13_phi_Vmax_upper_err = np.array(xmmlss13_phi_Vmax) + np.array(xmmlss13_phi_Vmax_err)
xmmlss13_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss13_phi_Vmax))) - np.abs(np.log10(xmmlss13_phi_Vmax_upper_err)))

xmmlss13_phi_Vmax_lower_err = np.array(xmmlss13_phi_Vmax) - np.array(xmmlss13_phi_Vmax_err)
xmmlss13_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss13_phi_Vmax))) - np.abs(np.log10(xmmlss13_phi_Vmax_lower_err)))
xmmlss13_phi_Vmax_lower_err_size_log = np.where(xmmlss13_phi_Vmax_lower_err_size_log == float('inf'), 20,
                                            xmmlss13_phi_Vmax_lower_err_size_log)

<ipython-input-270-3a735d63b92f>:28: RuntimeWarning: divide by zero encountered in log10
  xmmlss13_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss13_phi_Vmax))) - np.abs(np.log10(xmmlss13_phi_Vmax_lower_err)))


In [271]:
'''XMMLSS14'''
# List to store 1/Vmax bins for W50s
xmmlss14_phi_Vmax = []
xmmlss14_phi_Vmax_err = []

xmmlss14_bin_id_count = 1 # Counter for bin ID

while xmmlss14_bin_id_count < len(w50_bins):
    # Sum for bin 
    xmmlss14_sum_bin_Vmax = 0
    xmmlss14_sum_bin_Vmax_err = 0
    
    for i in range(len(xmmlss14_bin_id)):
        if xmmlss14_bin_id[i] == xmmlss14_bin_id_count:
            xmmlss14_sum_bin_Vmax += 1/xmmlss14_Vmax_list[i]
            xmmlss14_sum_bin_Vmax_err += pow(1/xmmlss14_Vmax_list[i], 2)

    xmmlss14_phi_Vmax.append(xmmlss14_sum_bin_Vmax)
    xmmlss14_phi_Vmax_err.append(np.sqrt(xmmlss14_sum_bin_Vmax_err))
    
    # Increment bin ID counter
    xmmlss14_bin_id_count += 1
    
xmmlss14_phi_Vmax_upper_err = np.array(xmmlss14_phi_Vmax) + np.array(xmmlss14_phi_Vmax_err)
xmmlss14_phi_Vmax_upper_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss14_phi_Vmax))) - np.abs(np.log10(xmmlss14_phi_Vmax_upper_err)))

xmmlss14_phi_Vmax_lower_err = np.array(xmmlss14_phi_Vmax) - np.array(xmmlss14_phi_Vmax_err)
xmmlss14_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss14_phi_Vmax))) - np.abs(np.log10(xmmlss14_phi_Vmax_lower_err)))
xmmlss14_phi_Vmax_lower_err_size_log = np.where(xmmlss14_phi_Vmax_lower_err_size_log == float('inf'), 20,
                                            xmmlss14_phi_Vmax_lower_err_size_log)

<ipython-input-271-99a1691182fe>:28: RuntimeWarning: divide by zero encountered in log10
  xmmlss14_phi_Vmax_lower_err_size_log = np.abs(np.abs(np.log10(np.array(xmmlss14_phi_Vmax))) - np.abs(np.log10(xmmlss14_phi_Vmax_lower_err)))


For comparison, let us add the modified Schechter functions from Zwaan 2010 and Papastergis 2011.

In [272]:
def schechter_zwaan(W, phistar, Wstar, alpha, beta):
    return np.log(10)*phistar*pow((W/Wstar), alpha)*np.exp(-pow((W/Wstar), beta))*(beta/sp.special.gamma(alpha/beta))

In [273]:
# Parameters used in Zwaan 2010's paper
phi_star = 0.67
W_star = pow(10, 2.21)
alpha_zwaan = 0.10
beta_zwaan = 1.56

zwaan_w = np.linspace(1, pow(10, w50_max)+500, 1500)
zwaan_schechter = schechter_zwaan(zwaan_w, phi_star, W_star, alpha_zwaan, beta_zwaan)

In [274]:
def schechter_papastergis(w, phistar, wstar, alpha, beta):
    return np.log(10)*phistar*pow((w/wstar), alpha)*np.exp(-pow((w/wstar), beta))

In [275]:
# Parameters used in Papastergis 2011 paper
pap_phi_star = 0.011
pap_w_star = pow(10, 2.58)
pap_alpha = -0.85
pap_beta = 2.7

pap_w = np.linspace(1, pow(10, w50_max)+500, 1500)
pap_schechter = schechter_papastergis(pap_w, pap_phi_star, pap_w_star, pap_alpha, pap_beta)

## Plotting the combined VFs


In [ ]:
# Definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom - 0.2, width, 0.2]

# start with a rectangular Figure
plt.figure(figsize=(15, 10))

# Scatter plot axes set up
ax_scatter = plt.axes(rect_scatter)
ax_scatter.xaxis.set_minor_locator(AutoMinorLocator())
ax_scatter.yaxis.set_minor_locator(AutoMinorLocator())
ax_scatter.axes.tick_params(axis='both', which='minor', length=5, direction='in', labelsize=24, top=True, right=True)
ax_scatter.axes.tick_params(axis='both', which='major', length=10, direction='in', labelsize=24, top=True, right=True)
ax_scatter.axes.tick_params(axis='x', pad=20)
ax_scatter.tick_params(axis='both', labelsize=24, labeltop=False, labelbottom=False)
ax_scatter.xaxis.set_label_position('top')
ax_scatter.set_xlabel(r'$W_{50}$ [km s$^{-1}$]', fontsize=24, labelpad=40)
ax_scatter.set_ylabel(r'log ($\phi$/h$^3$Mpc$^{-3}$)', fontsize=24)
#ax_scatter.set_title(r'Zwaan et al. (2010) $\frac{1}{V_\mathrm{max}}$ method', fontsize=20)

# Number of galaxies vs log w
ax_histx = plt.axes(rect_histx)
ax_histx.xaxis.set_minor_locator(AutoMinorLocator())
ax_histx.yaxis.set_minor_locator(AutoMinorLocator())
ax_histx.axes.tick_params(axis='x', which='minor', length=3, direction='in', labelsize=24, top=True, bottom=True)
ax_histx.axes.tick_params(axis='x', which='major', length=8, direction='in', labelsize=24, top=True, bottom=True)
ax_histx.axes.tick_params(axis='y', which='minor', length=5, direction='in', labelsize=24, right=True)
ax_histx.axes.tick_params(axis='y', which='major', length=10, direction='in', labelsize=24, right=True)
ax_histx.tick_params(direction='in', labelbottom=True, labeltop=False)
ax_histx.xaxis.set_label_position('bottom')
ax_histx.set_xlabel(r'log ($W_{50}$/km s$^{-1}$)', fontsize=24, labelpad=15)
ax_histx.set_ylabel(r'$N_\mathrm{gal}$', fontsize=24)
ax_histx.tick_params(axis='both', labelsize=24)

# Velocity Function
'''MIGHTEE'''
# yerr=0.434*np.array(mightee_phi_Vmax_err)/mightee_phi_Vmax
ax_scatter.plot(mightee_bin_centres[3:], np.log10(mightee_phi_Vmax)[3:], 'o', color='black', markersize=12, label=r'MIGHTEE-HI', alpha=0.8)
ax_scatter.errorbar(mightee_bin_centres[3:], np.log10(mightee_phi_Vmax)[3:], yerr=[mightee_phi_Vmax_lower_err_size_log[3:], mightee_phi_Vmax_upper_err_size_log[3:]], 
            capsize=5, fmt='', marker=None, ls='none', color='black', alpha=0.8)
ax_scatter.plot(mightee_bin_centres[0:3], np.log10(mightee_phi_Vmax)[0:3], 'o', color='black', markerfacecolor='white', markersize=12, label=None, alpha=1)
ax_scatter.errorbar(mightee_bin_centres[0:3], np.log10(mightee_phi_Vmax)[0:3], yerr=[mightee_phi_Vmax_lower_err_size_log[0:3], mightee_phi_Vmax_upper_err_size_log[0:3]], 
            capsize=5, fmt='', marker=None, ls='none', color='black', alpha=0.8)
    
''' COSMOS'''
ax_scatter.plot(cosm_bin_centres[3:], np.log10(cosm_phi_Vmax)[3:], 'o', color='royalblue', markersize=12, label=r'COSMOS', alpha=0.8)
ax_scatter.errorbar(cosm_bin_centres[3:], np.log10(cosm_phi_Vmax)[3:], yerr=[cosm_phi_Vmax_lower_err_size_log[3:], cosm_phi_Vmax_upper_err_size_log[3:]], 
            capsize=5, fmt='', marker=None, ls='none', color='royalblue', alpha=0.8)
ax_scatter.plot(cosm_bin_centres[0:3], np.log10(cosm_phi_Vmax)[0:3], 'o', color='royalblue', markersize=12, markerfacecolor='white', label=None, alpha=1)
ax_scatter.errorbar(cosm_bin_centres[0:3], np.log10(cosm_phi_Vmax)[0:3], yerr=[cosm_phi_Vmax_lower_err_size_log[0:3], cosm_phi_Vmax_upper_err_size_log[0:3]], 
            capsize=5, fmt='', marker=None, ls='none', color='royalblue', alpha=0.8)
    
'''XMMLSS'''
ax_scatter.plot(xmmlss_bin_centres[3:], np.log10(xmmlss_phi_Vmax)[3:], 'o', color='red', markersize=12, label=r'XMM-LSS', alpha=0.8)
ax_scatter.errorbar(xmmlss_bin_centres[3:], np.log10(xmmlss_phi_Vmax)[3:], yerr=[xmmlss_phi_Vmax_lower_err_size_log[3:], xmmlss_phi_Vmax_upper_err_size_log[3:]], 
            capsize=5, fmt='', marker=None, ls='none', color='red', alpha=0.8)
ax_scatter.plot(xmmlss_bin_centres[0:3], np.log10(xmmlss_phi_Vmax)[0:3], 'o', color='red', markersize=12, markerfacecolor='white', label=None, alpha=1)
ax_scatter.errorbar(xmmlss_bin_centres[0:3], np.log10(xmmlss_phi_Vmax)[0:3], yerr=[xmmlss_phi_Vmax_lower_err_size_log[0:3], xmmlss_phi_Vmax_upper_err_size_log[0:3]], 
            capsize=5, fmt='', marker=None, ls='none', color='red', alpha=0.8)

# '''XMMLSS12'''
# ax_scatter.plot(xmmlss12_bin_centres, np.log10(xmmlss12_phi_Vmax), 's', color='salmon', markersize=8, label=r'XMM-LSS 12', alpha=0.6)
# ax_scatter.errorbar(xmmlss12_bin_centres, np.log10(xmmlss12_phi_Vmax), yerr=[xmmlss12_phi_Vmax_lower_err_size_log, xmmlss12_phi_Vmax_upper_err_size_log], 
#             capsize=5, fmt='', marker=None, ls='none', color='salmon', alpha=0.6)
    
# '''XMMLSS13'''
# ax_scatter.plot(xmmlss13_bin_centres, np.log10(xmmlss13_phi_Vmax), 's', color='chocolate', markersize=8, label=r'XMM-LSS 13', alpha=0.6)
# ax_scatter.errorbar(xmmlss13_bin_centres, np.log10(xmmlss13_phi_Vmax), yerr=[xmmlss13_phi_Vmax_lower_err_size_log, xmmlss13_phi_Vmax_upper_err_size_log], 
#             capsize=5, fmt='', marker=None, ls='none', color='chocolate', alpha=0.6)
    
# '''XMMLSS14'''
# ax_scatter.plot(xmmlss14_bin_centres, np.log10(xmmlss14_phi_Vmax), 's', color='maroon', markersize=8, label=r'XMM-LSS 14', alpha=0.6)
# ax_scatter.errorbar(xmmlss14_bin_centres, np.log10(xmmlss14_phi_Vmax), yerr=[xmmlss14_phi_Vmax_lower_err_size_log, xmmlss14_phi_Vmax_upper_err_size_log], 
#             capsize=5, fmt='', marker=None, ls='none', color='maroon', alpha=0.6)

# '''Mightee schechter line'''
# ax_scatter.plot(np.log10(multinest_w50_linspace), np.log10(multinest_phi_mightee), '-', color='black', label='modified Schechter fit')

# '''Schechter lines from papers'''
# ax_scatter.plot(np.log10(zwaan_w), np.log10(zwaan_schechter), '--', color='lightskyblue', label='HIPASS fit')
# ax_scatter.plot(np.log10(pap_w), np.log10(pap_schechter), '--', color='palegreen', label='ALFALFA fit')

# Limits
ax_scatter.set_xlim((w50_min-0.143, w50_max+0.15))
ax_scatter.set_ylim((-4.8, 1.))

# Histograms
ax_histx.hist([mightee_w50, cosm_w50, xmmlss_w50], bins=w50_bins, stacked=False, rwidth=0.9, edgecolor='black', color=['black', 'royalblue', 'red'])
#ax_histx.hist([mightee_w50], bins=w50_bins, stacked=False, rwidth=1, edgecolor='black', color=['black'], fill=False)

# Label each bin
for rect in ax_histx.patches:
    height = rect.get_height()
    ax_histx.annotate(f'{int(height)}', xy=(rect.get_x()+rect.get_width()/2, height), 
                xytext=(0.5, 5), textcoords='offset points', ha='center', va='bottom', fontsize=12.2) 

# Tick positions of the twin x axis
ax_histx.set_ylim((0, 69))
ax_histx.set_xlim(ax_scatter.get_xlim())

# Tick positions of the twin x axis
# Set up the second axis
ax_scatter2 = ax_scatter.twiny()
ax_scatter2.axes.tick_params(axis='both', which = 'major', length=10, direction='in')
ax_scatter2.axes.tick_params(axis='x', which = 'minor', length=5, direction='in')

zticks = np.array([30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900])
zticks_labels = ['', '40', '', '60', '', '80', '', '100', '200', '300', '400', '500', '', '700', '800', '900']
ax_scatter2.set_xticks(func(zticks))
ax_scatter2.set_xticklabels(tick_function(zticks_labels))
ax_scatter_bounds = ax_scatter.get_xbound()
ax_scatter2.set_xlim(ax_scatter_bounds[0], ax_scatter_bounds[1])
ax_scatter2.axes.tick_params(labelsize=24)

# Legend
ax_scatter.legend(loc='lower left', fontsize=24, frameon=False)

plt.savefig(dirName+'all_fields_uncorrected_vf_mightee.pdf', bbox_inches='tight', dpi=300)

plt.show()

### Writing results to text file

In [299]:
# Save the output of the Zwaan method
# Create the text files
'''MIGHTEE'''
zwaan_mightee_vf = open(dirName+'/zwaan_mightee_vf.txt', 'w+')
for i in range(len(mightee_bin_centres)):
    zwaan_mightee_vf.write('%f %f %f\n'%(mightee_bin_centres[i], mightee_phi_Vmax[i], mightee_phi_Vmax_err[i]))
zwaan_mightee_vf.close()

'''COSMOS'''
zwaan_cosmos_vf = open(dirName+'/zwaan_cosmos_vf.txt', 'w+')
for i in range(len(cosm_bin_centres)):
    zwaan_cosmos_vf.write('%f %f %f\n'%(cosm_bin_centres[i], cosm_phi_Vmax[i], cosm_phi_Vmax_err[i]))
zwaan_cosmos_vf.close()

'''XMMLSS'''
zwaan_xmmlss_vf = open(dirName+'/zwaan_xmmlss_vf.txt', 'w+')
for i in range(len(xmmlss_bin_centres)):
    zwaan_xmmlss_vf.write('%f %f %f\n'%(xmmlss_bin_centres[i], xmmlss_phi_Vmax[i], xmmlss_phi_Vmax_err[i]))
zwaan_xmmlss_vf.close()

'''XMMLSS12'''
zwaan_xmmlss12_vf = open(dirName+'/zwaan_xmmlss12_vf.txt', 'w+')
for i in range(len(xmmlss12_bin_centres)):
    zwaan_xmmlss12_vf.write('%f %f %f\n'%(xmmlss12_bin_centres[i], xmmlss12_phi_Vmax[i], xmmlss12_phi_Vmax_err[i]))
zwaan_xmmlss12_vf.close()

'''XMMLSS13'''
zwaan_xmmlss13_vf = open(dirName+'/zwaan_xmmlss13_vf.txt', 'w+')
for i in range(len(xmmlss13_bin_centres)):
    zwaan_xmmlss13_vf.write('%f %f %f\n'%(xmmlss13_bin_centres[i], xmmlss13_phi_Vmax[i], xmmlss13_phi_Vmax_err[i]))
zwaan_xmmlss13_vf.close()

'''XMMLSS14'''
zwaan_xmmlss14_vf = open(dirName+'/zwaan_xmmlss14_vf.txt', 'w+')
for i in range(len(xmmlss14_bin_centres)):
    zwaan_xmmlss14_vf.write('%f %f %f\n'%(xmmlss14_bin_centres[i], xmmlss14_phi_Vmax[i], xmmlss14_phi_Vmax_err[i]))
zwaan_xmmlss14_vf.close()

In [300]:
# Write W50s to a text file
'''MIGHTEE'''
zwaan_mightee_w50 = open(dirName+'/mightee_w50.txt', 'w+')
for i in range(len(mightee_w50)):
    zwaan_mightee_w50.write('%f\n'%(mightee_w50[i]))
zwaan_mightee_w50.close()

'''COSMOS'''
zwaan_cosmos_w50 = open(dirName+'/cosmos_w50.txt', 'w+')
for i in range(len(cosm_w50)):
    zwaan_cosmos_w50.write('%f\n'%(cosm_w50[i]))
zwaan_cosmos_w50.close()

'''XMMLSS'''
zwaan_xmmlss_w50 = open(dirName+'/xmmlss_w50.txt', 'w+')
for i in range(len(xmmlss_w50)):
    zwaan_xmmlss_w50.write('%f\n'%(xmmlss_w50[i]))
zwaan_xmmlss_w50.close()

'''XMMLSS12'''
zwaan_xmmlss12_w50 = open(dirName+'/xmmlss12_w50.txt', 'w+')
for i in range(len(xmmlss12_w50)):
    zwaan_xmmlss12_w50.write('%f\n'%(xmmlss12_w50[i]))
zwaan_xmmlss12_w50.close()

'''XMMLSS13'''
zwaan_xmmlss13_w50 = open(dirName+'/xmmlss13_w50.txt', 'w+')
for i in range(len(xmmlss13_w50)):
    zwaan_xmmlss13_w50.write('%f\n'%(xmmlss13_w50[i]))
zwaan_xmmlss13_w50.close()

'''XMMLSS14'''
zwaan_xmmlss14_w50 = open(dirName+'/xmmlss14_w50.txt', 'w+')
for i in range(len(xmmlss14_w50)):
    zwaan_xmmlss14_w50.write('%f\n'%(xmmlss14_w50[i]))
zwaan_xmmlss14_w50.close()